**EXERCISE:** Answer the following questions based on the Adult dataset
1. What is the mean age of all persons in the data?
1. What is the mean age of female persons? What about male persons?
1. How many different types of educations are there?
2. What are the different types of education and how many persons are the for each type?
3. Is there a difference in educational level across sex?
4. What is the most common relationship status?
5. Is there a correlation between hours per week (worked) and age?
6. Is the average hours per week (worked) different across different marital-status groups?
7. Is there an income difference across sexes?